<a href="https://colab.research.google.com/github/sundarayamrita/NLP/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
corpus = open('WarAndPeace.txt', 'rb').read().decode(encoding='utf-8')[1:]
corpus = corpus.replace(',', ' ')
corpus = corpus.replace('\"', ' ')
corpus = corpus.replace('!', '.')
corpus = corpus.replace('?', '.')

In [0]:
import re
def sentence_tokeniser(corpus):
  return list(corpus[7424:].split('.'))

sentences = sentence_tokeniser(corpus)

In [0]:
def tokenize_corpus(sentences):
    tokens = [x.split() for x in sentences]
    return tokens

tokenized_corpus = tokenize_corpus(sentences)

In [0]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [0]:
window_size = 2
idx_pairs = []
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    for center_word_pos in range(len(indices)):
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = torch.from_numpy(np.array(idx_pairs))

In [0]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [11]:
embedding_dims = 300
num_epochs = 100
learning_rate = 0.001
W_target = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W_context = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)

for epoch in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W_target, x)
        z2 = torch.matmul(W_context, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data[0]
        loss.backward()
        W_target.data -= learning_rate * W_target.grad.data
        W_context.data -= learning_rate * W_context.grad.data

        W_target.grad.data.zero_()
        W_context.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

IndexError: ignored

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        # self.conv1 = nn.Conv2d(1, 6, 3)
        # self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.W_word = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
        self.W_context = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
        #self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        #x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        h = F.(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features